<a href="https://colab.research.google.com/github/alexcarvajal/ModeloPrediccionBRT/blob/main/OptunaLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalación Optuna

In [2]:
!pip install Optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.7 MB/s eta 0:00:00


# Búsqueda de hiperparámetros Optuna

In [1]:
import optuna
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
import time
import psutil
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error, r2_score


# Carga de datos
url1 = '/content/drive/MyDrive/Reuniones/DatasetsWP2/Dataset1CodificadoV2.csv'
data1 = pd.read_csv(url1)
data1.drop(['Unnamed: 0'], axis=1, inplace=True)

# Preprocesamiento de datos
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data1)
X = scaled_data[:, data1.columns != 'Inputs']
y = scaled_data[:, data1.columns == 'Inputs']

# Función para reestructurar los datos para el LSTM
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X[i:(i + time_steps)]
        Xs.append(v)
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

# Definición de la función objetivo para Optuna
def objective(trial):
    time_steps = trial.suggest_int('time_steps', 1, 10)
    X_lstm, y_lstm = create_dataset(X, y, time_steps)
    X_train, X_test, y_train, y_test = train_test_split(X_lstm, y_lstm, test_size=0.2, random_state=42)

    model = Sequential()
    model.add(LSTM(units=trial.suggest_categorical('units', [50, 100, 150]),
                   return_sequences=trial.suggest_categorical('return_sequences', [True, False]),
                   input_shape=(X_train.shape[1], X_train.shape[2])))
    if trial.suggest_categorical('return_sequences', [True, False]):
        model.add(LSTM(units=trial.suggest_categorical('units_2', [50, 100, 150])))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer=trial.suggest_categorical('optimizer', ['adam', 'rmsprop']))

    # Entrenamiento del modelo
    model.fit(X_train, y_train, epochs=trial.suggest_int('epochs', 20, 50), batch_size=trial.suggest_categorical('batch_size', [16, 32, 64]), verbose=3)
    y_pred = model.predict(X_test)

    # Cálculo de métricas
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)  # Cálculo del RMSE
    mae = mean_absolute_error(y_test, y_pred)  # Cálculo del MAE
    r2 = r2_score(y_test, y_pred)  # Cálculo del R^2

    # Mostrar las métricas después de cada trial
    print(f'Trial done: RMSE={rmse}, MAE={mae}, R^2={r2}')
    print(f'Params: {trial.params}')

    return mse

# Crear un estudio Optuna y optimizar
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=3)  # Puedes aumentar n_trials para una búsqueda más exhaustiva

# Imprimir los mejores hiperparámetros
print('Mejores hiperparámetros:', study.best_trial.params)
print('Mejor resultado:', study.best_value)


# Puedes construir y evaluar el modelo con los mejores hiperparámetros encontrados
# Siguiendo un proceso similar al descrito anteriormente, pero utilizando los hiperparámetros sugeridos por Optuna


[I 2024-03-06 19:23:27,903] A new study created in memory with name: no-name-bc5c45ed-9453-485a-aa24-f26442e4de5d


Epoch 1/25
Epoch 2/25
Epoch 3/25
Epoch 4/25
Epoch 5/25
Epoch 6/25
Epoch 7/25
Epoch 8/25
Epoch 9/25
Epoch 10/25
Epoch 11/25
Epoch 12/25
Epoch 13/25
Epoch 14/25
Epoch 15/25
Epoch 16/25
Epoch 17/25
Epoch 18/25
Epoch 19/25
Epoch 20/25
Epoch 21/25
Epoch 22/25
Epoch 23/25
Epoch 24/25
Epoch 25/25
2734/2734 [==============================] - 14s 5ms/step


[I 2024-03-06 20:21:17,368] Trial 0 finished with value: 0.0019124780899126641 and parameters: {'time_steps': 6, 'units': 100, 'return_sequences': False, 'optimizer': 'adam', 'epochs': 25, 'batch_size': 16}. Best is trial 0 with value: 0.0019124780899126641.


Trial done: RMSE=0.043731888707357065, MAE=0.024089816103941192, R^2=0.8411712527963783
Params: {'time_steps': 6, 'units': 100, 'return_sequences': False, 'optimizer': 'adam', 'epochs': 25, 'batch_size': 16}
Epoch 1/38
Epoch 2/38
Epoch 3/38
Epoch 4/38
Epoch 5/38
Epoch 6/38
Epoch 7/38
Epoch 8/38
Epoch 9/38
Epoch 10/38
Epoch 11/38
Epoch 12/38
Epoch 13/38
Epoch 14/38
Epoch 15/38
Epoch 16/38
Epoch 17/38
Epoch 18/38
Epoch 19/38
Epoch 20/38
Epoch 21/38
Epoch 22/38
Epoch 23/38
Epoch 24/38
Epoch 25/38
Epoch 26/38
Epoch 27/38
Epoch 28/38
Epoch 29/38
Epoch 30/38
Epoch 31/38
Epoch 32/38
Epoch 33/38
Epoch 34/38
Epoch 35/38
Epoch 36/38
Epoch 37/38
Epoch 38/38
2734/2734 [==============================] - 10s 3ms/step


[I 2024-03-06 21:45:54,747] Trial 1 finished with value: 0.003436431166017825 and parameters: {'time_steps': 3, 'units': 150, 'return_sequences': False, 'optimizer': 'adam', 'epochs': 38, 'batch_size': 16}. Best is trial 0 with value: 0.0019124780899126641.


Trial done: RMSE=0.058621081242312696, MAE=0.03217994728995857, R^2=0.7151627994735325
Params: {'time_steps': 3, 'units': 150, 'return_sequences': False, 'optimizer': 'adam', 'epochs': 38, 'batch_size': 16}
Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50
Epoch 20/50
Epoch 21/50
Epoch 22/50
Epoch 23/50
Epoch 24/50
Epoch 25/50
Epoch 26/50
Epoch 27/50
Epoch 28/50
Epoch 29/50
Epoch 30/50
Epoch 31/50
Epoch 32/50
Epoch 33/50
Epoch 34/50
Epoch 35/50
Epoch 36/50
Epoch 37/50
Epoch 38/50
Epoch 39/50
Epoch 40/50
Epoch 41/50
Epoch 42/50
Epoch 43/50
Epoch 44/50
Epoch 45/50
Epoch 46/50
Epoch 47/50
Epoch 48/50
Epoch 49/50
Epoch 50/50
2734/2734 [==============================] - 15s 5ms/step


[I 2024-03-06 23:04:39,937] Trial 2 finished with value: 0.0030316832461948612 and parameters: {'time_steps': 4, 'units': 150, 'return_sequences': True, 'units_2': 50, 'optimizer': 'rmsprop', 'epochs': 50, 'batch_size': 64}. Best is trial 0 with value: 0.0019124780899126641.


Trial done: RMSE=0.0550607232625477, MAE=0.03172763875357147, R^2=0.7503315421618515
Params: {'time_steps': 4, 'units': 150, 'return_sequences': True, 'units_2': 50, 'optimizer': 'rmsprop', 'epochs': 50, 'batch_size': 64}
Mejores hiperparámetros: {'time_steps': 6, 'units': 100, 'return_sequences': False, 'optimizer': 'adam', 'epochs': 25, 'batch_size': 16}
Mejor resultado: 0.0019124780899126641
